In [61]:
import torch
import cuml
from torch_geometric.data import Data

In [76]:
event = torch.load("/global/cfs/cdirs/m3443/data/ITk-upgrade/processed/gnn_processed/0GeV_v3/train/0001").cpu()

In [77]:
def get_performance(event, score_cut):
    mask = (event.scores > score_cut)
    edges = event.edge_index[:, mask]
    node_mask = torch.zeros(event.x.shape[0])
    node_mask[edges.unique()] = 1
    node_mask = node_mask.bool()
    signal_mask = (event.pt > 1000.) & (event.nhits >= 5) & (event.primary == 1)
    pid = event.pid.unique(return_inverse = True)[1]
    nhits = event.pid.unique(return_counts = True)[1]
    hit_eff = (pid[node_mask & signal_mask].unique(return_counts = True)[1] > 0.5*nhits[pid[node_mask & signal_mask].unique()]).sum()/\
                pid[signal_mask].unique().shape[0]
    eff = event.y[mask].sum()/signal_mask[event.modulewise_true_edges].all(0).sum()
    pur = event.y_pid[mask].sum()/event.y_pid[mask].shape[0]
    
    return hit_eff, eff, pur, node_mask.sum(), edges.shape[1]

In [78]:
get_performance(event, 0.1)

(tensor(0.9877), tensor(0.9534), tensor(0.6426), tensor(50009), 68606)

Data()